In [115]:
import pickle
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import sklearn.tree as tree
import sklearn.ensemble as ensemble

In [135]:
data = pickle.load(open("Data/TRAININGDATA.SALAMP.xlsx K3-Part0.p", "rb"))
data2 = pickle.load(open("Data/TRAININGDATA.SALAMP.xlsx K3-Part1.p", "rb"))
# data2["Antibiotic"] = "Test"
df = data.append(data2)

# Change columns names of MIC in case there are similar column names
df.columns = [v for v in df.columns[:1]] + ["MIC_val"] + [v for v in df.columns[2:]]

In [149]:
df

,Antibiotic,MIC_val,EVW,MIP,PPT,AFG,KPH,CTN,CTK,PQX,...,FCA,RDW,TLY,AWR,VCE,LLH,GST,PVN,RAA,AMP
0,AMP,0.0,71.0,114.0,112.0,506.0,52.0,21.0,23.0,0.0,...,74.0,79.0,278.0,190.0,74.0,345.0,639.0,210.0,892.0,1
1,AMP,0.0,70.0,107.0,117.0,500.0,56.0,25.0,18.0,0.0,...,73.0,78.0,281.0,189.0,71.0,329.0,672.0,214.0,893.0,1
2,AMP,0.0,72.0,114.0,113.0,512.0,51.0,21.0,23.0,0.0,...,74.0,79.0,278.0,191.0,74.0,345.0,643.0,211.0,905.0,1
3,AMP,0.0,69.0,107.0,122.0,474.0,59.0,20.0,18.0,0.0,...,74.0,79.0,302.0,179.0,64.0,333.0,642.0,216.0,882.0,1
4,AMP,0.0,78.0,107.0,109.0,463.0,53.0,23.0,17.0,0.0,...,74.0,75.0,289.0,182.0,67.0,334.0,681.0,212.0,854.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,AMP,6.0,76.0,111.0,106.0,505.0,62.0,23.0,20.0,0.0,...,77.0,81.0,306.0,200.0,72.0,338.0,726.0,219.0,936.0,1
2634,AMP,6.0,71.0,107.0,111.0,509.0,65.0,22.0,18.0,0.0,...,77.0,80.0,304.0,195.0,67.0,328.0,723.0,228.0,891.0,1
2635,AMP,0.0,68.0,111.0,113.0,488.0,52.0,24.0,17.0,0.0,...,75.0,84.0,307.0,189.0,66.0,326.0,608.0,209.0,850.0,1
2636,AMP,6.0,71.0,104.0,115.0,489.0,54.0,25.0,20.0,0.0,...,76.0,85.0,284.0,189.0,72.0,356.0,666.0,217.0,877.0,1


In [136]:
# Create 1 hot encoding columns and append to the end
dummies = pd.get_dummies(df.Antibiotic)
df = pd.concat([df, dummies], axis=1)

In [142]:
# Get input and ouput columns
X = df.drop(columns=["Antibiotic", "MIC_val"], axis=1)
y = df.iloc[:,1]

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision tree

In [148]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7386363636363636

In [ ]:
clf = tree.DecisionTreeRegressor()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

# Random Forest

In [101]:
clf = ensemble.RandomForestClassifier(n_estimators=100, max_features="auto",random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.834280303030303

In [ ]:
DecisionTreeRegressor

# AdaBoost

In [102]:
clf = ensemble.AdaBoostClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

KeyboardInterrupt: 

# Gradient Boosting

In [ ]:
clf = ensemble.GradientBoostingClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, [y_pred-1, y_pred+1])

In [110]:
df.iloc[:,1] = 

ValueError: Must have equal len keys and value when setting with an iterable